In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from keras import backend as K
from sklearn.metrics import make_scorer
from tensorflow.keras.models import Sequential
import keras
from tensorflow.keras.layers import Dense
from keras.optimizers import Adam 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
from scipy.stats import randint

In [ ]:
train_450 = pd.read_csv('ACT1_train_450.csv')
train_anova = pd.read_csv('ACT1_Anova_50.csv')
train_50 = pd.read_csv('ACT1_train_50_corr_red.csv') 

In [ ]:
#450
y_450 = train_450['Act'].values
train_450 = train_450.drop(['Act', 'Unnamed: 0'], axis = 1)
x_450 = train_450.values
X_train_450, X_test_450, Y_train_450, Y_test_450 = train_test_split(x_450, y_450, test_size = 0.20, random_state = 0)
#X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=0)

In [ ]:
#anova
y_anova = train_anova['Act'].values
train_anova = train_anova.drop(['Act', 'Unnamed: 0'], axis = 1)
x_anova = train_anova.values
X_train_anova, X_test_anova, Y_train_anova, Y_test_anova = train_test_split(x_anova, y_anova, test_size = 0.20, random_state = 0)


In [ ]:
#50
y_50 = train_50['Act'].values
train_50 = train_50.drop(['Act', 'Unnamed: 0'], axis = 1)
x_50 = train_50.values
X_train_50, X_test_50, Y_train_50, Y_test_50 = train_test_split(x_50, y_50, test_size = 0.20, random_state = 0)


In [ ]:
Y_train_450 = np.reshape(Y_train_450,(len(Y_train_450),1))
Y_test_450 = np.reshape(Y_test_450,(len(Y_test_450),1))
Y_train_anova = np.reshape(Y_train_anova,(len(Y_train_anova),1))
Y_test_anova = np.reshape(Y_test_anova,(len(Y_test_anova),1))
Y_train_50 = np.reshape(Y_train_50,(len(Y_train_50),1))
Y_test_50 = np.reshape(Y_test_50,(len(Y_test_50),1))


In [ ]:
def input_dimension(array):
    return np.shape(array)[1]

In [ ]:
def r_square(y_pred, y_true):
    """ r^2 value defined by the competition host, r^2 = 1 indicates 100% prediction accuracy
    """
    avx = np.mean(y_pred)
    avy = np.mean(y_true)
    sum1, sumx, sumy = 0, 0, 0
    for i in range(len(y_pred)):
        sum1 += (y_pred[i] - avx)*(y_true[i] - avy)
        sumx += (y_pred[i] - avx)*(y_pred[i] - avx)
        sumy += (y_true[i] - avy)*(y_true[i] - avy)
    print(len(y_pred), sum1, sumx, sumy)
    return sum1*sum1/(sumx*sumy)

In [ ]:
#Keras R_2 function
R_2 = []
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
#Keras MAPE function
MAPE = []
def mean_ape(y_true, y_pred):
    return K.mean(K.abs((y_true-y_pred)/y_true))*100
#Scikit learn mape function
def mean_apes(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))*100
def mean_ae(y_true,y_pred):
    return np.mean(np.abs(y_true-y_pred))

In [ ]:
mape_func = make_scorer(mean_apes)
r2_func = make_scorer(r_square)
mae_func = make_scorer(mean_ae)

In [ ]:
def build_model_450(learn_rate):
    model = keras.models.Sequential();
    model.add(keras.layers.core.Dense(units = 50, activation='relu', input_dim = input_dimension(X_train_450)));
    model.add(keras.layers.core.Dense(units= 25, activation = 'relu'))
    model.add(keras.layers.core.Dense(1, activation = 'relu'));
    adam = Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False);
    model.compile(optimizer=adam,loss='mean_squared_error')
    return model

In [ ]:
def build_model_50(learn_rate):
    model = keras.models.Sequential();
    model.add(keras.layers.core.Dense(units = 40, activation='relu', input_dim = input_dimension(X_train_50)));
    model.add(keras.layers.core.Dense(units=20, activation = 'relu'))
    model.add(keras.layers.core.Dense(1, activation = 'relu'));
    adam = Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False);
    model.compile(optimizer=adam,loss='mean_squared_error')
    return model

In [ ]:
def build_model_anova(learn_rate):
    model = keras.models.Sequential();
    model.add(keras.layers.core.Dense(units = 40, activation='relu', input_dim = input_dimension(X_train_anova)));
    model.add(keras.layers.core.Dense(units=20, activation = 'relu'))
    model.add(keras.layers.core.Dense(1, activation = 'relu'));
    adam = Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False);
    model.compile(optimizer=adam,loss='mean_squared_error')
    return model

In [ ]:
regressor_50 = KerasRegressor(build_fn=build_model_50)
regressor_anova = KerasRegressor(build_fn=build_model_anova)
regressor_450 = KerasRegressor(build_fn=build_model_450)

In [ ]:
scoring = {'R2':r2_func, 'MAE': mae_func, 'MAPE': mape_func}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameters ={'batch_size':randint(50,200),
            'nb_epoch': randint(50,500),
            'learn_rate':[0.001,0.01,0.1]}
random_search_50 = RandomizedSearchCV(estimator=regressor_50, param_distributions=parameters, cv=5, scoring = scoring, n_iter=10, refit = 'MAPE', return_train_score = True, pre_dispatch = 2)
random_search_anova = RandomizedSearchCV(estimator=regressor_anova, param_distributions=parameters, cv=5, scoring = scoring, n_iter= 100, refit = 'MAPE', return_train_score = True, pre_dispatch = 2)
random_search_450 = RandomizedSearchCV(estimator=regressor_450, param_distributions=parameters, cv=5, scoring = scoring, n_iter=100, refit = 'MAPE', return_train_score = True, pre_dispatch = 2)

In [ ]:
random_search_50.fit(X_train_50, Y_train_50)

In [ ]:
random_search_anova.fit(X_train_anova, Y_train_anova)

In [ ]:
random_search_450.fit(X_train_450, Y_train_450)

In [ ]:
result_50 = pd.DataFrame(random_search_50.cv_results_)
result_anova = pd.DataFrame(random_search_anova.cv_results_)
result_450 = pd.DataFrame(random_search_450.cv_results_)

In [ ]:
result_50.to_csv('../Results/RandomSearch_Tuning_ANN_iter100_50.csv')

In [ ]:
result_anova.to_csv('../Results/RandomSearch_Tuning_ANN_iter100_anova.csv')

In [ ]:
result_450.to_csv('../Results/RandomSearch_Tuning_ANN_iter100_450.csv')